In [1]:
import calendar
import datetime as dt
from itertools import chain
from datetime import datetime, timedelta
from IPython.core import display as ICD
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

In [2]:
# -- STORE ALL FITBIT DATA INTO LISTS OF SHEET --
xls_files = glob.glob('Fitbit/*.xls')

## Functions

In [3]:
def add_weekday(df, not_index=0):
    """Add day of week column"""
    
    # -- USE DATAFRAME'S INDEX IF 'not_index' = 1, OTHERWISE USE DATAFRAME'S 'Date' COLUMN --
    col = df.index
    if not_index:
        col = df['Date']
    
    # -- DETERMINE THE DAY OF WEEK FROM THE DATE AND STORE IN A LIST (USED TO ADD AS A COLUMN TO THE DATAFRAME) --
    weekday = []
    for date in col:
        #weekday.append(datetime.strptime(str(date), '%Y-%m-%d').strftime('%A'))
        weekday.append(calendar.day_name[date.weekday()])
    df.loc[:, 'Weekday'] = weekday
    
    
def to_num(df, list_of_cols):
    """Remove commas and convert values to a numeric data type"""
    
    for col in list_of_cols:
        #df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('\D', '', regex=True)
        df[col] = pd.to_numeric(df[col])
        

def to_date(df, list_of_col):
    """Convert columns into datetime type"""
    
    for col in list_of_col:
        df[col] = pd.to_datetime(df[col])

### Function for cleaning sleep data

In [4]:
def sort_sleep_data(list_of_df):
    """Sort sleep data by date"""
    
    for i in range(len(list_of_df)):
        list_of_df[i].sort_values(by='Start Time', ascending=True, inplace=True)

### Functions for wrangling food and macros data

In [5]:
def flatten_list(l):
    """Flatten a list"""
    
    return list(chain(*l))


def get_sheetnames_and_dates(excel_file):
    """Only get sheet names and dates for days where food data is entered"""
    """Returns: 2 lists (sheet names & dates)"""
    
    monthly_calories = pd.ExcelFile(excel_file).parse('Foods')

    dates = []
    sheet_names = []
    
    # -- GET SHEET NAMES AND DATES ONLY FOR THE DAYS WHERE INFO ABOUT FOOD INTAKE IS ENTERED --
        # -- daily[0]: date --
        # -- daily[1]: calories --
    for daily in monthly_calories.values:
        if str(daily[1]) != '0':
            dates.append(datetime.strptime(daily[0], '%Y-%m-%d').date())
            foodLogSheetName = 'Food Log ' + daily[0].replace('-', '')
            sheet_names.append(foodLogSheetName)
    return sheet_names, dates


def remove_empty_rows(xls, sheet_list):
    """Converting data in labeled table format to dataframe format"""
    """Returns: dataframe without unnecessary data"""
    
    df_list = [pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_list]

    # -- REMOVING UNNECESSARY ROWS & FILLING IN 'MISSING' DATA --
    for df in df_list:
        df.dropna(how='all', inplace=True)
        df['Meal'] = df['Meal'].fillna(method='ffill')
        df.reset_index(drop=True, inplace=True)
        
    return df_list


def get_food_and_macros(food_list, dates_list):
    """Get macros information into desirable dataset format"""
    """Returns: 2 dataframes (one with food consumption data & one with daily macros info)"""

    macros_df = pd.DataFrame()
    all_food_df = pd.DataFrame()
    all_macros_df = pd.DataFrame()
    
    # -- SPLITTING DATA INTO TWO DIFFERENT DATAFRAMES --
    for ind, (df, date) in enumerate(zip(food_list, dates_list)):
        
        index_of_macros = []

        # -- STORE INDEX OF ROWS THAT HAVE MACROS DATA --
        for i in df.index:
            if df['Meal'][i] == 'Daily Totals':
                index_of_macros.append(i)

        # -- EXTRACT MACROS DATA FROM THE DATAFRAME. CREATE & STORE MACROS DATA IN A NEW DATAFRAME --
        macros_df = df.iloc[index_of_macros[1]:index_of_macros[-1]+1]
        macros_df['Date'] = date
        macros_df = macros_df.pivot(index='Date', columns='Food', values='Calories')
        all_macros_df = all_macros_df.append(macros_df)

        # -- ADD 'Date' COLUMN TO THE FOOD DATAFRAME --
        food_list[ind]['Date'] = date

        # -- STORE EACH UPDATED FOOD DATAFRAME INTO A NEW DATAFRAME --
        all_food_df = all_food_df.append(food_list[ind].drop(index=index_of_macros).dropna())

    return all_food_df, all_macros_df

## Data wrangling

### Cleaning activities and sleep data

In [6]:
# -- STORING DATA IN EXCEL AS LISTS DATAFRAMES --
list_of_activities_df = [pd.ExcelFile(xls).parse('Activities', index_col=0, parse_dates=True) for xls in xls_files]
list_of_sleep_df = [pd.ExcelFile(xls).parse('Sleep', usecols=':F') for xls in xls_files]

# -- CONCATING ALL FOOD DATAFRAMES INTO ONE BIG DATAFRAME --
activities = pd.concat(list_of_activities_df)

# -- CONCATING ALL SLEEP DATAFRAMES INTO ONE BIG DATAFRAME --
sort_sleep_data(list_of_sleep_df)
sleep = pd.concat(list_of_sleep_df)
sleep.reset_index(drop=True, inplace=True)

# -- CLEANING ACTIVITIES DATAFRAME (REMOVE MISSING VALUES & REPLACING OUTLIERS WITH MEAN) --
to_num(activities, ['Calories Burned', 'Steps', 'Minutes Sedentary', 'Activity Calories'])
activities = activities[activities['Steps'] != 0]

for col in activities.columns:
    mean = activities[col].mean()
    std = activities[col].std()
    activities[col] = activities[col].mask(((activities[col] - mean).abs() > 2 * std), mean)

# -- ADD DAY OF WEEK COLUMN --
add_weekday(activities)

### Cleaning food data

In [7]:
# -- GET ALL NECESSARY SHEET NAMES AND DATES --
sheet_names, dates = zip(*[get_sheetnames_and_dates(xls) for xls in xls_files])
dates = [d for d in dates if d]

# -- FIRST STEP IN DATA CLEANING: REMOVING ALL EMPTY ROWS --
list_of_food_df = [remove_empty_rows(xls, monthly_sheets) for xls, monthly_sheets in zip(xls_files, sheet_names) if monthly_sheets]

# -- SECOND STEP IN DATA CLEANING: SPLIT ORIGINAL DATAFRAMES INTO SEPARATE DATAFRAMES --
list_of_food_df, list_of_macros_df = zip(*[get_food_and_macros(monthly_food, date) for monthly_food, date in zip(list_of_food_df, dates)])

macros = pd.concat(list_of_macros_df)
food = pd.concat(list_of_food_df)
food.reset_index(drop=True, inplace=True)

# -- ADD DAY OF WEEK COLUMN --
add_weekday(macros)
add_weekday(food, 1)

/Users/vwu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Activities data

In [8]:
sns.set()

In [9]:
activities

,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,Weekday
Date,,,,,,,,,,
2015-10-21,2150.000000,14061.000000,5.710000,17.000000,531.452381,324.599426,0.000000,0.000000,1566.116635,Wednesday
2015-10-22,2274.000000,13617.000000,5.460000,12.000000,596.000000,300.000000,17.000000,69.000000,1344.000000,Thursday
2015-10-23,2174.000000,16530.000000,6.570000,20.000000,639.000000,361.000000,15.000000,35.000000,1275.000000,Friday
2015-10-24,2161.000000,14710.000000,5.880000,11.000000,550.000000,278.000000,36.000000,52.000000,1227.000000,Saturday
2015-10-25,2459.466539,16972.642447,2.020000,8.000000,869.000000,324.599426,9.000000,14.000000,1566.116635,Sunday
2015-10-26,2097.000000,13642.000000,5.490000,19.000000,847.000000,254.000000,40.000000,44.000000,1136.000000,Monday
2015-10-27,1978.000000,11329.000000,4.600000,13.000000,684.000000,251.000000,16.000000,39.000000,989.000000,Tuesday
2015-10-28,2171.000000,13462.000000,5.370000,16.000000,724.000000,318.000000,11.000000,55.000000,1251.000000,Wednesday
2015-10-29,2306.000000,13626.000000,5.460000,19.000000,643.000000,342.000000,24.000000,61.000000,1407.000000,Thursday


## Sleep data

In [10]:
sleep

,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed
0,2015-10-22 12:00AM,2015-10-22 5:07AM,292,15,1,307
1,2015-10-22 9:29PM,2015-10-23 4:17AM,401,7,1,408
2,2015-10-23 9:47PM,2015-10-24 6:43AM,514,22,2,536
3,2015-10-24 11:24PM,2015-10-25 7:16AM,459,13,1,472
4,2015-10-24 2:40PM,2015-10-24 4:05PM,80,5,0,85
5,2015-10-26 1:48PM,2015-10-26 3:54PM,116,10,1,126
6,2015-10-26 9:51PM,2015-10-27 5:10AM,416,23,1,439
7,2015-10-27 9:40PM,2015-10-28 5:32AM,449,23,1,472
8,2015-10-29 10:53PM,2015-10-30 4:00AM,299,8,1,307
9,2015-10-29 12:09AM,2015-10-29 5:12AM,293,10,1,303


In [11]:
# -- CONVERT VALUES TO DATETIME --
to_date(sleep, ['Start Time', 'End Time'])

In [12]:
dates = []

# -- USING START TIME TO DETERMINE WHICH DAY THE INSTANCE OF SLEEP SHOULD BE ASSIGNED --
for i in range(len(sleep['Start Time'])):
    date = sleep['Start Time'][i].date()
    time = sleep['Start Time'][i].time()
    if time > dt.time(4):
        dates.append(date)
    else:
        dates.append(date - timedelta(1))
sleep['Date'] = dates

# -- GROUP ALL SLEEP DATA BY DAY --
daily_sleep = sleep.groupby('Date').sum()
daily_sleep.index = pd.to_datetime(daily_sleep.index)

# -- ADD DAY OF WEEK COLUMN --
add_weekday(daily_sleep)

daily_sleep

,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Weekday
Date,,,,,
2015-10-21,292,15,1,307,Wednesday
2015-10-22,401,7,1,408,Thursday
2015-10-23,514,22,2,536,Friday
2015-10-24,539,18,1,557,Saturday
2015-10-26,532,33,2,565,Monday
2015-10-27,449,23,1,472,Tuesday
2015-10-28,293,10,1,303,Wednesday
2015-10-29,299,8,1,307,Thursday
2015-10-30,520,32,1,552,Friday


## Food

In [13]:
food

,Meal,Food,Calories,Date,Weekday
0,Breakfast,American Cheese,61,2015-11-09,Monday
1,Breakfast,"Bagel thins, Everything",110,2015-11-09,Monday
2,Breakfast,"Egg, Chicken, Fried",184,2015-11-09,Monday
3,Breakfast,"Ham Steak, Traditional",30,2015-11-09,Monday
4,Morning Snack,Dark Chocolate Dreams,170,2015-11-09,Monday
5,Morning Snack,Banana,90,2015-11-09,Monday
6,Morning Snack,"Rice Cakes, Salt Free",70,2015-11-09,Monday
7,Breakfast,"English Muffin, Original",129,2015-11-11,Wednesday
8,Breakfast,"Egg, Chicken, Fried",184,2015-11-11,Wednesday
9,Breakfast,Bacon Pre-Cooked (S),75,2015-11-11,Wednesday


## Macros

In [14]:
macros

Food,Calories,Carbs,Fat,Fiber,Protein,Sodium,Water,Weekday
Date,,,,,,,,
2015-11-09,715,72 g,34 g,8 g,35 g,943 mg,0 fl oz,Monday
2015-11-11,797,74 g,39 g,4 g,37 g,"1,064 mg",0 fl oz,Wednesday
2015-11-12,"1,049",108 g,45 g,11 g,53 g,"1,216 mg",0 fl oz,Thursday
2015-11-30,90,20 g,0 g,1 g,1 g,2 mg,0 fl oz,Monday
2015-12-02,240,29 g,6 g,3 g,17 g,152 mg,0 fl oz,Wednesday
2015-12-09,860,101 g,35 g,8 g,37 g,"1,105 mg",0 fl oz,Wednesday
2015-12-10,"1,054",135 g,40 g,26 g,58 g,"1,210 mg",0 fl oz,Thursday
2015-12-11,"1,157",155 g,35 g,23 g,68 g,679 mg,0 fl oz,Friday
2015-12-15,"1,162",142 g,44 g,13 g,57 g,"1,402 mg",0 fl oz,Tuesday
